In [1]:
def extract_scramble_list(filepath):
    with open(filepath) as f:
        lines = f.readlines()

    scramble_list = []
    for l in lines:
        scramble = l.strip().split('.')[1][1:]
        scramble_list.append(scramble)
    
    return scramble_list

# scramble_list = extract_scramble_list("999_F2L_scrambles.txt")

In [2]:
list_scramble = extract_scramble_list("15000_full_scrambles.txt")
len(list_scramble)

15000

In [18]:
%%time
import requests
import json
from tqdm import tqdm


# url = "https://ai-cube-api.herokuapp.com/solve_cube"
url = "http://127.0.0.1:8000/solve_cube"


success = []
start = 200
nb_test = 1000
good_scramble = []
bad_scramble = []
err = {"cross_errors": 0, "f2l_errors": 0, "oll_errors": 0, "pll_errors": 0}
f2l_errors = {"F2L1": 0, "F2L2": 0, "F2L3": 0, "F2L4": 0}
cross_errors = {"daisy": 0, "cross_from_daisy":0}
for i in tqdm(range(nb_test)): 
    scramble = {"scramble_s": list_scramble[i+start]}
    result = requests.post(url=url, json=scramble).json()
    correct = result['solution_found']
    success.append(correct)
    # print(scramble)
    if correct:
        good_scramble.append(list_scramble[i+start])
    # else: 
    #     print(result['step_solutions'])
    else:
        bad_scramble.append(list_scramble[i+start])
        error = result["step_solutions"]
        if error['Cross'][0] == "No":
            err["cross_errors"] += 1
            if error['Cross'][1] == "Daisy":
                cross_errors['daisy'] +=1
            elif error['Cross'][1] == "Cross":
                cross_errors['cross_from_daisy'] +=1
        elif error["F2L"][0] == "No":
            if error['F2L'][1] == "F2L1":
                f2l_errors['F2L1'] +=1
            elif error['F2L'][1] == "F2L2":
                f2l_errors['F2L2'] +=1
            elif error['F2L'][1] == "F2L3":
                f2l_errors['F2L3'] +=1
            elif error['F2L'][1] == "F2L4":
                f2l_errors['F2L2'] +=1
            err["f2l_errors"] += 1
        elif error["OLL"][0] == "No":
            err["oll_errors"] += 1
        elif error["PLL"][0] == "No":
            err["pll_errors"] += 1

    if i%nb_test//10 == 0 and i>0: print(f'\nsuccess {sum(success)}/{i}')

print(f'Success rate {sum(success)/nb_test*100:.2f}%')
print(err)
print(cross_errors)
print(f2l_errors)

100%|██████████| 1000/1000 [39:25<00:00,  2.37s/it]Success rate 65.70%
{'cross_errors': 129, 'f2l_errors': 207, 'oll_errors': 0, 'pll_errors': 7}
Wall time: 39min 25s

